In [ ]:
import glob
import pandas
import matplotlib.pyplot as plt
import seaborn
import json

In [ ]:
outdir = "../outputs/plots/"

In [ ]:
!ls /local/joosep/ml-tau-en-reg/results

In [ ]:
losses = {}
losses["jet_regression"] = {
    "LorentzNet": json.load(open("/local/joosep/ml-tau-en-reg/results/240524_cosinescheduler/jet_regression/LorentzNet/history.json")),
    "SimpleDNN": json.load(open("/local/joosep/ml-tau-en-reg/results/240524_cosinescheduler/jet_regression/SimpleDNN/history.json")),
    "ParticleTransformer": json.load(open("/local/joosep/ml-tau-en-reg/results/240524_cosinescheduler/jet_regression/ParticleTransformer/history.json")),
}

losses["dm_multiclass"] = {
    "LorentzNet": json.load(open("/local/joosep/ml-tau-en-reg/results/240524_cosinescheduler/dm_multiclass/LorentzNet/history.json")),
    "SimpleDNN": json.load(open("/local/joosep/ml-tau-en-reg/results/240524_cosinescheduler/dm_multiclass/SimpleDNN/history.json")),
    "ParticleTransformer": json.load(open("/local/joosep/ml-tau-en-reg/results/240528_PT_num_layers_4/dm_multiclass/ParticleTransformer/history.json")),
}

In [ ]:
# training_type = "jet_regression"
# training_title = "$p_T$ regression"
# ylim = 0.02

training_type = "dm_multiclass"
training_title = "Decay mode classification"
ylim = 1.0


In [ ]:
c = plt.plot(losses[training_type]["ParticleTransformer"]["losses_train"], ls="--", lw=1)[0]
plt.plot(losses[training_type]["ParticleTransformer"]["losses_validation"], label="ParticleTransformer", color=c.get_color())

c = plt.plot(losses[training_type]["LorentzNet"]["losses_train"], ls="--", lw=1)[0]
plt.plot(losses[training_type]["LorentzNet"]["losses_validation"], label="LorentzNet", color=c.get_color())

c = plt.plot(losses[training_type]["SimpleDNN"]["losses_train"], ls="--", lw=1)[0]
plt.plot(losses[training_type]["SimpleDNN"]["losses_validation"], label="DeepSet", color=c.get_color())

plt.ylim(0.0, ylim)
plt.legend(loc="best", title="Validation")
plt.ylabel("Loss")
plt.xlabel("Training epoch")
plt.title(training_title)
plt.savefig(outdir + "/loss_{}.pdf".format(training_type))